# Data Analytics Project - Preprocessing
# Initial Data Cleaning and processing

---

# 1. Importing required modules

In [1]:
import os
import pandas as pd

---

# 2. Extracting CSV files from folders

In [2]:
companies_folders = os.listdir("../Storage/Companies")

print()
print("Number of companies folders:", len(companies_folders))

# Checking if this step was completed earlier
if not os.path.exists("../Storage/Companies_csvs/"):

    # Making a folder of all csvs
    os.mkdir("../Storage/Companies_csvs/")

    # Extracting csvs present in each companies folder into companies_csv folder for ease of use
    for company_folder in companies_folders:
        # print(company_folder)
        if os.path.isdir(f"../Storage/Companies/{company_folder}"):
            for csv_file in os.listdir(f"../Storage/Companies/{company_folder}"):
                os.rename(f"../Storage/Companies/{company_folder}/{csv_file}", f"../Storage/Companies_csvs/{csv_file}")
                # print("        " + csv_file)
            os.rmdir(f"../Storage/Companies/{company_folder}")

companies_csv_files = os.listdir("../Storage/Companies_csvs")

print("After processing ...")
print("Number of companies csv files:", len(companies_csv_files))
print()



Number of companies folders: 3792
After processing ...
Number of companies csv files: 4993



---

# 3. Handling missing data

We are dropping rows of the datasets which have missing data.

Interpolation would not work in this scenario as filling in the data for missing days may lead to discrepancies and might affect the mean reverting nature of larger stocks and increase volatility.


In [3]:
# In each dataset, we are dropping rows with NaN values

companies_csvs = os.listdir("../Storage/Companies_csvs")

print("Number of companies csv files:", len(companies_csvs))

if not os.path.exists("../Storage/Companies_drop_rows/"):

    # Making a folder with cleaned csvs
    os.mkdir("../Storage/Companies_drop_rows/")

    # Dropping rows of all csvs with NaN values
    for company_csv in companies_csvs:
        df = pd.read_csv(f"../Storage/Companies_csvs/{company_csv}", index_col=[0])
        df = df.dropna()
        df.to_csv(f"../Storage/Companies_drop_rows/{company_csv}")

    # For debuging purposes only
    # os.rmdir("../Storage/Companies_drop_rows/")

companies_drop_rows = os.listdir("../Storage/Companies_drop_rows")

print("After processing ...")

print("Number of companies csv files cleaned:", len(companies_drop_rows))


Number of companies csv files: 4993
After processing ...
Number of companies csv files cleaned: 4993


---

# 4. Cropping all datasets to be in the range of 2017-2019 (Included)

In [4]:
# In each dataset, we are checking if there is adequate data and 
# removing data that doesn't fit in the range of years 2017-2019

companies_drop_rows = os.listdir("../Storage/Companies_drop_rows/")

print()

print("Number of companies csv files with dropped rows:", len(companies_drop_rows))

if not os.path.exists("../Storage/Companies_in_range/"):

    # Making a folder with csvs in above specified range
    os.mkdir("../Storage/Companies_in_range/")

    # Dropping rows of all csvs with NaN values
    for cleaned_company_csv in companies_drop_rows:
        df = pd.read_csv(f"../Storage/Companies_drop_rows/{cleaned_company_csv}", index_col=[0])
        # print()
        # print(cleaned_company_csv)
        if df.shape[0] > 1000:
            # print(df.head())
            dates_in_range = list(map(lambda date: date[:4] in ["2017", "2018", "2019"], df.index))
            df = df[dates_in_range]
            # print(df.head())
            df.to_csv(f"../Storage/Companies_in_range/{cleaned_company_csv}")

    # For debuging purposes only
    # os.rmdir("../Storage/Companies_in_range/")

companies_in_range = os.listdir("../Storage/Companies_in_range/")

print("After processing ...")

print("Number of companies csv files cleaned:", len(companies_in_range))

print()




Number of companies csv files with dropped rows: 4993
After processing ...
Number of companies csv files cleaned: 3558



---

# 5. Adding Company name and Exchange columns

In [5]:
# In each dataset, we are adding the company name and exchange columns
# to make it easier to identify rows in the dataframes later

companies_in_range = os.listdir("../Storage/Companies_in_range/")

print()

print("Number of companies csv files with dropped rows:", len(companies_in_range))

if not os.path.exists("../Storage/Companies_with_names_exchange/"):

    # Making a folder with csvs in above specified range
    os.mkdir("../Storage/Companies_with_names_exchange/")

    # Dropping rows of all csvs with NaN values
    for company_in_range_csv in companies_in_range:
        df = pd.read_csv(f"../Storage/Companies_in_range/{company_in_range_csv}", index_col=[0])
        # print()
        # print(cleaned_company_csv)
#         print()
#         print("csv: ", company_in_range_csv)
#         print("Company name: ", company_in_range_csv[:-7])
#         print("Company exchange: ", company_in_range_csv[-7:-4])
        df["Company"] = company_in_range_csv[:-7]
        df["Exchange"] = company_in_range_csv[-7:-4]
        # print(df.head())
        df.to_csv(f"../Storage/Companies_with_names_exchange/{company_in_range_csv}")

    # For debuging purposes only
    # os.rmdir("../Storage/Companies_with_names_exchange/")

companies_with_names_exchange = os.listdir("../Storage/Companies_with_names_exchange/")

print("After processing ...")

print("Number of companies csvs with names and exchanges added:", len(companies_with_names_exchange))

print()



Number of companies csv files with dropped rows: 3558
After processing ...
Number of companies csvs with names and exchanges added: 3558



---

# 6. Grouping stocks by sector

## 6.1 Reading csv with all stock tickers

In [6]:
equity = pd.read_csv('../Storage/Equity.csv')
equity.head()

,Security Code,Issuer Name,Security Id,Security Name,Status,Group,Face Value,ISIN No,Industry,Instrument
0,500002,NaN,ABB,ABB India Limited,Active,B,2.00,INE117A01022,Heavy Electrical Equipment,Equity
1,500003,NaN,AEGISLOG,AEGIS LOGISTICS LTD.,Active,A,1.00,INE208C01025,Oil Marketing & Distribution,Equity
2,500004,NaN,TPAEC,TORRENT POWER AEC LTD.,Delisted,B,10.00,INE424A01014,,Equity
3,500005,NaN,AKARLAMIN,AKAR LAMINATORS LTD.,Delisted,XD,10.00,INE984C01013,Iron & Steel Products,Equity
4,500006,NaN,ALPHADR,ALPHA DRUG INDIA LTD.,Delisted,B,10.00,INE256B01026,,Equity


## 6.2 Filtering the dataset to include only equities

In [7]:
equity = equity[equity['Instrument'] == 'Equity']
equity.head()

,Security Code,Issuer Name,Security Id,Security Name,Status,Group,Face Value,ISIN No,Industry,Instrument
0,500002,NaN,ABB,ABB India Limited,Active,B,2.00,INE117A01022,Heavy Electrical Equipment,Equity
1,500003,NaN,AEGISLOG,AEGIS LOGISTICS LTD.,Active,A,1.00,INE208C01025,Oil Marketing & Distribution,Equity
2,500004,NaN,TPAEC,TORRENT POWER AEC LTD.,Delisted,B,10.00,INE424A01014,,Equity
3,500005,NaN,AKARLAMIN,AKAR LAMINATORS LTD.,Delisted,XD,10.00,INE984C01013,Iron & Steel Products,Equity
4,500006,NaN,ALPHADR,ALPHA DRUG INDIA LTD.,Delisted,B,10.00,INE256B01026,,Equity


## 6.3 Remove tickers where industry (sector) is not mentioned

In [8]:
equity = equity[equity['Industry'].str.strip() != '']
equity.head()

,Security Code,Issuer Name,Security Id,Security Name,Status,Group,Face Value,ISIN No,Industry,Instrument
0,500002,NaN,ABB,ABB India Limited,Active,B,2.00,INE117A01022,Heavy Electrical Equipment,Equity
1,500003,NaN,AEGISLOG,AEGIS LOGISTICS LTD.,Active,A,1.00,INE208C01025,Oil Marketing & Distribution,Equity
3,500005,NaN,AKARLAMIN,AKAR LAMINATORS LTD.,Delisted,XD,10.00,INE984C01013,Iron & Steel Products,Equity
5,500008,NaN,AMARAJABAT,AMARA RAJA BATTERIES LTD.,Active,A,1.00,INE885A01032,Auto Parts & Equipment,Equity
6,500009,NaN,AMBALALSA,AMBALAL SARABHAI ENTERPRISES LTD.,Active,X,10.00,INE432A01017,Pharmaceuticals,Equity


## 6.4 Filter tickers to include only actively trading stocks

In [9]:
equity = equity[equity['Status'] == 'Active']
equity.head()

,Security Code,Issuer Name,Security Id,Security Name,Status,Group,Face Value,ISIN No,Industry,Instrument
0,500002,NaN,ABB,ABB India Limited,Active,B,2.00,INE117A01022,Heavy Electrical Equipment,Equity
1,500003,NaN,AEGISLOG,AEGIS LOGISTICS LTD.,Active,A,1.00,INE208C01025,Oil Marketing & Distribution,Equity
5,500008,NaN,AMARAJABAT,AMARA RAJA BATTERIES LTD.,Active,A,1.00,INE885A01032,Auto Parts & Equipment,Equity
6,500009,NaN,AMBALALSA,AMBALAL SARABHAI ENTERPRISES LTD.,Active,X,10.00,INE432A01017,Pharmaceuticals,Equity
7,500010,NaN,HDFC,HOUSING DEVELOPMENT FINANCE CORP.LTD.,Active,A,2.00,INE001A01036,Housing Finance,Equity


## 6.5 Dropping irrelevant columns

In [10]:
equity = equity.drop(columns=['Security Code', 'Issuer Name', 'Status', 'Group', 'Face Value', 'ISIN No', 'Instrument'])
equity.head()

,Security Id,Security Name,Industry
0,ABB,ABB India Limited,Heavy Electrical Equipment
1,AEGISLOG,AEGIS LOGISTICS LTD.,Oil Marketing & Distribution
5,AMARAJABAT,AMARA RAJA BATTERIES LTD.,Auto Parts & Equipment
6,AMBALALSA,AMBALAL SARABHAI ENTERPRISES LTD.,Pharmaceuticals
7,HDFC,HOUSING DEVELOPMENT FINANCE CORP.LTD.,Housing Finance


## 6.6 Renaming columns, setting index and converting to csv

In [11]:
equity = equity.rename(columns={"Security Id": "Symbol", "Security Name": "Company", "Industry": "Sector"})
equity

,Symbol,Company,Sector
0,ABB,ABB India Limited,Heavy Electrical Equipment
1,AEGISLOG,AEGIS LOGISTICS LTD.,Oil Marketing & Distribution
5,AMARAJABAT,AMARA RAJA BATTERIES LTD.,Auto Parts & Equipment
6,AMBALALSA,AMBALAL SARABHAI ENTERPRISES LTD.,Pharmaceuticals
7,HDFC,HOUSING DEVELOPMENT FINANCE CORP.LTD.,Housing Finance
...,...,...,...
8345,SHARIABEES,NIPPON INDIA ETF SHARIAH BEES,Other Financial Services
8346,QNIFTY,QUANTUM MUTUAL FUND - QUANTUM INDEX FUND ETF,Other Financial Services
8350,M50,Motilal Oswal MOSt Shares M50 ETF,Other Financial Services
8356,ASHIKACR,ASHIKA CREDIT CAPITAL LTD.,Finance (including NBFCs)


In [12]:
equity = equity.set_index('Symbol')
equity.head()

,Company,Sector
Symbol,,
ABB,ABB India Limited,Heavy Electrical Equipment
AEGISLOG,AEGIS LOGISTICS LTD.,Oil Marketing & Distribution
AMARAJABAT,AMARA RAJA BATTERIES LTD.,Auto Parts & Equipment
AMBALALSA,AMBALAL SARABHAI ENTERPRISES LTD.,Pharmaceuticals
HDFC,HOUSING DEVELOPMENT FINANCE CORP.LTD.,Housing Finance


In [13]:
equity.to_csv('../Storage/stocks_by_sectors.csv')